In [2]:
from pathlib import Path
import networkx as nx
from collections import defaultdict
from collections import Counter
from tqdm import tqdm
import pickle as pk
import numpy as np
import pandas as pd
from datetime import datetime as dt
from geopy.distance import great_circle
import matplotlib.pyplot as plt
from random import choices
import itertools
from multiprocessing import Pool
import geopandas as gpd
import time
import gc
import bisect
import sys
sys.path.append("/")
import homophily_functions as fn

%matplotlib inline



In [19]:
import importlib
importlib.reload(fn)
importlib.reload(gpd)

<module 'geopandas' from '/home/quint/general_venv/lib/python3.12/site-packages/geopandas/__init__.py'>

CALCULATE POPULATION IN BETWEEN EACH TIE OF THE NETWORK

In [4]:
tracts = gpd.read_file("../Aux_Data/Tract_2010Census_DP1/Tract_2010Census_DP1.shp")
tracts = tracts.to_crs('ESRI:102003')
tracts = tracts[["GEOID10", "DP0010001", "geometry"]]

In [ ]:
G = pk.load(open("../Pickles/follower_graph.pk", "rb"))    
distance = nx.get_edge_attributes(G,"distance")
fips = nx.get_node_attributes(G, "fips")
del G

In [9]:
distance_l = sorted(list(distance.items()), key = lambda tup: tup[0][0])
distance_g = {key:{} for key in set([x[0][0] for x in distance_l])}
for edge, dist in distance_l:
    distance_g[edge[0]][edge] = dist

In [ ]:
#radiation_pop = {edge:None for edge in distance.keys()}
def get_pop(inp):
    send = inp[0]
    d = inp[1]
    out = {edge:None for edge in d}
    cur_tract = tracts.loc[tracts.GEOID10 == fips[send], "geometry"]
    if cur_tract.shape[0]>0:
        for edge, dist in list(d.items()): 
            if not dist == None:
                radius = cur_tract.centroid.buffer(dist*1000).values[0]
                intersect = tracts.iloc[tracts.sindex.query(radius, "intersects")]
                out[edge] = sum(intersect.DP0010001)
            else:
                out[edge] = None
    else:
        return(None)
    return(out)

pool = Pool(30)
radiation_pop = pool.map(get_pop, list(distance_g.items()))
pool.close()

with open("Pickles/radiation_pop.pk", "wb") as f:
    pk.dump(radiation_pop, f)

In [ ]:
radiation_pop = {key:val for d in radiation_pop if d!=None for key,val in d.items()}
with open("Pickles/radiation_pop.pk", "wb") as f:
    pk.dump(radiation_pop, f)

CALCULATE POPULATION IN BETWEEN RANDOM SAMPLE OF DYADS

In [3]:
tracts = gpd.read_file("../Aux_Data/Tract_2010Census_DP1/Tract_2010Census_DP1.shp")
tracts = tracts.to_crs('ESRI:102003')
tracts = tracts[["GEOID10", "DP0010001", "geometry"]]

In [4]:
G = pk.load(open("../Pickles/follower_graph.pk", "rb"))    

In [5]:
# this subsets the node keys that do not has na for coords and also filters out HI and Ak
lat = nx.get_node_attributes(G,"lat")
lon = nx.get_node_attributes(G,"lon")
nodes_rem = [key for key,val in lat.items() if pd.isna(val) or pd.isna(lon[key]) or G.nodes()[key]['state'] in ['HI', 'AK']]
G.remove_nodes_from(nodes_rem)

In [20]:
distance_r = fn.generate_random_distance(G, len(G.edges()))

100%|████████████████████████████████████████████████████████████████████████████████| 17409685/17409685 [03:05<00:00, 93696.06it/s]


In [8]:
fips = nx.get_node_attributes(G, "fips")

In [21]:
distance_l = sorted(list(distance_r.items()), key = lambda tup: tup[0][0])
distance_g = {key:{} for key in set([x[0][0] for x in distance_l])}
for edge, dist in distance_l:
    distance_g[edge[0]][edge] = dist

In [22]:
def get_pop(inp):
    send = inp[0]
    d = inp[1]
    out = {edge:None for edge in d}
    cur_tract = tracts.loc[tracts.GEOID10 == fips[send], "geometry"]
    if cur_tract.shape[0]>0:
        for edge, dist in list(d.items()): 
            if not dist == None:
                radius = cur_tract.centroid.buffer(dist*1000).values[0]
                intersect = tracts.iloc[tracts.sindex.query(radius, "intersects")]
                out[edge] = sum(intersect.DP0010001)
            else:
                out[edge] = None
    else:
        return(None)
    return(out)


In [23]:
with Pool(60) as p:
    radiation_pop = list(tqdm(p.imap(get_pop, list(distance_g.items())), total = len(distance_g)))

radiation_pop = {key:val for d in radiation_pop if d!=None for key,val in d.items()}


 43%|███████████████████████████████████                                               | 447901/1045651 [8:02:02<7:32:50, 22.00it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [24]:
len(radiation_pop)

17408560

In [26]:
with open("../Pickles/radiation_pop_rand_labels_2.pk", "wb") as f:
    pk.dump(radiation_pop, f)